# Creating the Machine Learning Model to train the model's detection of emotion

In [1]:
import pandas as pd
import numpy as np

In [2]:
ed = pd.read_csv("emotion_sentimen_dataset.csv")
ed.head()

,Unnamed: 0,text,Emotion
0,0,i seriously hate one subject to death but now ...,hate
1,1,im so full of life i feel appalled,neutral
2,2,i sit here to write i start to dig out my feel...,neutral
3,3,ive been really angry with r and i feel like a...,anger
4,4,i feel suspicious if there is no one outside l...,neutral


In [3]:
from sklearn.model_selection import train_test_split
print("Done")
from sklearn.preprocessing import StandardScaler
print("Done")
import tensorflow as tf
print("Done")
from keras.layers import Dense
print("Done")
from keras.models import Sequential
print("Done")

Done
Done


: 